In [1]:
import torch
import torchtext
from torch.utils.data import DataLoader

train_data, test_data = torchtext.datasets.AG_NEWS()

In [2]:

from torchtext.data import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

TOKENIZER = get_tokenizer('basic_english')

def build_vocabulary(datasets):
    for dataset in datasets:
        for _, text in dataset:
            yield (TOKENIZER(text))

vocab = build_vocab_from_iterator(build_vocabulary([train_data, test_data]), min_freq=1, specials=['<unk>'])
vocab.set_default_index(vocab['<unk>'])


In [3]:
tokens = TOKENIZER('some text for test of tokenezation')
indexes = vocab(tokens)
print(indexes, tokens)

[122, 4003, 11, 287, 6, 0] ['some', 'text', 'for', 'test', 'of', 'tokenezation']


In [5]:
from torchtext.data.functional import to_map_style_dataset
import torch
from torch.utils.data import DataLoader

MAX_WORDS = 50

def vectorize_batch(batch):
    X, Y = list(zip(*batch))
    X = [vocab(TOKENIZER(text) for text in X)]
    X = [tokens ([0]*(MAX_WORDS-len(tokens))) if len(tokens)<MAX_WORDS else tokens[:MAX_WORDS] for tokens in X]
    return torch.tensor(X, dtype=torch.int32, device='cpu'), torch.tensor(Y, device='cpu') - 1

train_data, test_data = to_map_style_dataset(train_data), to_map_style_dataset(test_data)
train_loader = DataLoader(train_data, batch_size=64, collate_fn=vectorize_batch, shuffle=True)
test_loader = DataLoader(test_data, batch_size=256, collate_fn=vectorize_batch)

In [ ]:
from torch import nn
from torch.nn import functional as F
import gensim.downloader

embed_dim = 300
hedden_dim = 70
n_layers = 1

class RNNClassifier(nn.Module):
    def init_embed(self):
        model = gensim.downloader.load('glove-wiki-gigaword-300')
        weights = torch.FloatTensor(model.vectors)
        self.embedding_layer = nn.Embedding.from_pretrained(weights)

    def __init__(self):
        super(RNNClassifier, self).__init__()
        self.rnn = nn.RNN(input_size=embed_dim, hidden_size=hedden_dim, num_layers=n_layers, batch_first=True)
        # здесь лучше взять LSTM или GRU ячейки
        self.lienar = nn.Linear(hedden_dim, 4) #4 - количество классов
        self.init_embed()

    def forward(self, X_batch):
        emb = self.embedding_layer
        output, (hedden, _) = self.rnn(emb)
        return self.lienar(output[:,-1])

In [ ]:
for epoch in range():
    for batch in x_train:
        preds = model(batch)
        loss = (T, preds)
        loss.backwards()
        optimizer.step()
        optimizer.zero_grad()